In [2]:
import requests 
from bs4 import BeautifulSoup

import pandas as pd
import re
from time import sleep

from csv import writer

In [4]:
def limpiar_fila(fila):

    columns=["KM","Año","Combustible","Ciudad","Nº puertas", "Garantía","Cambio","Color","Carrocería","Marca", "Modelo", "Versión","Potencia", "Tipo de carrocería", "Velocidad máxima","Aceleracion","Consumo","Emisiones","Dimensiones","Etiqueta medioambiental"]
    nueva_fila = {}

    for column in columns:
        
        nueva_fila[column] = False

        for i in range(len(fila)):
            
            if column == "Carrocería":
                if ": " not in fila[i] and "\n" not in fila[i]:
                    nueva_fila["Carrocería"] = fila[i]
                    break


            if fila[i][0:len(column)] == column:

                dato = fila[i].split(": ")[-1]
                dato = dato.split("\n")[-1]
                nueva_fila[column] = dato
                break
        
        if column == "Potencia" and not nueva_fila[column]:

            nueva_fila[column] == re.findall(r"(\w+)CV", fila[-1])
             
            if not nueva_fila[column]:
                nueva_fila[column] = None

    
    #nueva_fila.append(fila[8])
    nueva_fila["Precio"] = fila[-2]

    return nueva_fila

In [5]:
def obtener_datos(url):

    response = requests.get(url)

    print(response)

    print(bool(response))

    soup = BeautifulSoup(response.text, "html.parser")
    div = soup.find("div", class_="panel-body")
    datos1 = [i.text for i in div.find_all("li")]
    datos1.append(div.find_all("ul")[1].text.split(" ")[1])
    datos2 = [i.text.strip() for i in soup.find_all("tr")]

    datos = datos1 + datos2
    
    precio = soup.find("div", class_="precio").text.strip()
    datos.append(precio)
    
    description = soup.find("div", id="textoDescripcionCoche").text
    datos.append(description)
    
    return datos

In [6]:
lista_enlaces_total = []

#Este es el for para obtener los enlaces de las páginas, en el range poner pagina inicial y final 
for i in range(500,1000):


    response = requests.get(f"https://coches.km77.com/ultimos-anuncios?page={i}")

    print(response)

    print(bool(response))

    soup = BeautifulSoup(response.text, "html.parser")

    lista_tr = soup.find_all("tr")
    lista_enlaces = []
    for tr in lista_tr:
        lista_enlaces.append("https://coches.km77.com"+tr.find("a")["href"])

    lista_enlaces_total += lista_enlaces

    sleep(2)
    
print(f"Enlaces obtnidos: {len(lista_enlaces_total)}")

<Response [200]>
True


KeyboardInterrupt: 

In [ ]:
datos_obtenidos = []
for url in lista_enlaces_total[1:]:

    try:

        datos = obtener_datos(url)
    
    except:
        try:
            sleep(5)
            datos = obtener_datos(url)
        except:

            continue

    try:
    
        datos_limpios = limpiar_fila(datos)
        datos_obtenidos.append(list(datos_limpios.values()))
    
    except:
    
        continue


<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response [200]>
True
<Response 

In [ ]:
cvs = pd.DataFrame(data = datos_obtenidos,columns=['KM','Año','Combustible','Ciudad','Nº puertas','Garantía','Cambio','Color','Carrocería','Marca','Modelo','Versión','Potencia','Tipo de carrocería' ,'Velocidad máxima' ,'Aceleracion','Consumo','Emisiones','Dimensiones','Etiqueta medioambiental' ,'Precio'])
cvs.to_csv("./datos_km_77.csv", index= False)

In [ ]:
df = pd.read_csv("./datos_km_77.csv")

In [ ]:
df["Precio"].unique().shape

(2597,)